In [5]:
import os
# os.environ["CUDA_VISIBLE_DEVICES"]="0" # second gpu

#### Task 2
import pickle
import numpy as np
import time
from matplotlib import pyplot as plt
from matplotlib.ticker import FuncFormatter
from matplotlib.ticker import MultipleLocator, AutoMinorLocator
from scipy import interpolate
from scipy.stats import shapiro
from scipy.stats import normaltest
from scipy.stats import anderson
from sklearn.preprocessing import MinMaxScaler

def TMSD(traj, t_lags):
    ttt = np.zeros_like(t_lags, dtype= float)
    for idx, t in enumerate(t_lags): 
        for p in range(len(traj)-t):
            ttt[idx] += (traj[p]-traj[p+t])**2            
        ttt[idx] /= len(traj)-t    
    return ttt

def aging(traj, twind):
    age = np.zeros(len(twind))
    for i, it in enumerate(twind):
        traj_seg = traj[0:it]
        age[i] = TMSD(traj_seg,[1])[0]
    return age

def bound(val):
    if val >= 100:
        val = 100
    elif val <= -100:
        val = -100
    else:
        val = val
    return val

def trajec_feature(traces):
    ## scaling factor
    if np.mean(np.abs(np.diff(traces))) != 0:
        sfac = (np.max(traces)-np.min(traces))/np.mean(np.abs(np.diff(traces)))
    else:
        sfac = 0
    
    mean = bound( np.mean(traces) )
    ms = bound( np.std(traces) )
    m3 = 0 ; m4 = 0
    if ms != 0:
        m3 = bound( np.mean(np.power(traces-mean,3))/(ms**3) )
        m4 = bound( np.mean(np.power(traces-mean,4))/(ms**4) )



    vtraces = np.diff(traces)
    vmean = bound( np.mean(vtraces) )
    vms = bound( np.std(vtraces) )
    vm3 = 0 ; vm4 = 0
    if vms != 0:
        vm3 = bound( np.mean(np.power(vtraces-vmean,3))/(vms**3) )
        vm4 = bound( np.mean(np.power(vtraces-vmean,4))/(vms**4) )

    atraces = np.diff(vtraces)
    amean = bound( np.mean(atraces) )
    ams = bound( np.std(atraces) )
    am3 = 0 ; am4 = 0
    if ams != 0:
        am3 = bound( np.mean(np.power(atraces-amean,3))/(ams**3) )
        am4 = bound( np.mean(np.power(atraces-amean,4))/(ams**4) )


    f2 = bound( np.max(vtraces) - np.min(vtraces) )
    f3 = bound( np.max(atraces) - np.min(atraces) )
    
       
    tlag = np.linspace(1, len(traces)-1, 10, dtype = 'int')
    
    msd = TMSD(traces, tlag)
    A = np.vstack([np.log(tlag), np.ones(len(np.log(tlag)))]).T
    nw1, nw0 = np.linalg.lstsq(A, np.log(msd), rcond=None)[0]

    nw1 = bound(nw1)
    nw0 = bound(nw0)
    
    tlag = np.linspace(10, len(traces)-1, 10, dtype = 'int')
    age = aging(traces, tlag)
    A = np.vstack([np.log(tlag), np.ones(len(np.log(tlag)))]).T
    age_nw1, age_nw0 = np.linalg.lstsq(A, np.log(age), rcond=None)[0]
    
    age_nw1 = bound(age_nw1)

    sh_stat, sh_p = shapiro(vtraces)
    
    if sh_p >= 0.1:
        sh_stat = -1      
    sh_stat = bound(sh_stat)
    
    if len(vtraces) > 20 :
        nom_stat, nom_p = normaltest(vtraces)
        if nom_p >= 0.1:
            nom_stat = -1
        and_result = anderson(vtraces).statistic
    
    else:
        nom_stat = -0.5
        and_result = -0.5
       
    nom_stat = bound(nom_stat)
    and_result = bound(and_result)

    
    feature = np.asarray([mean, ms, m3, m4, vmean, vms, vm3, vm4, \
                        amean, ams, am3, nw1, nw0, age_nw1, \
                        sfac, am4, f2, f3, sh_stat, and_result])
    
    

    feature = np.append(feature,msd)
    feature = np.append(feature,age)

    
    

    return feature


In [14]:
file = '../USED_DATA/task_sim_1d_0_nmax5000.pkl'
nmax = 5000

f = open(file, 'rb')
my_list = pickle.load(f)

trajecs = my_list['trajec']
model_id = my_list['label_id']
labels = my_list['label']
elabels = my_list['elabel']

f.flush()
f.close()


ncase = int(len(trajecs)/nmax)


tstart = time.time()


ndiv = 100
namp = 0.01

data_set = np.zeros((len(trajecs),ndiv))
scaler = MinMaxScaler(feature_range=(0, 1))

for i in range(len(trajecs)):
    trajec = trajecs[i]
    ret = len(trajecs[i])
    convt = ndiv/ret
    
    ti_new = np.linspace(0, len(trajec), ndiv)
    postck = interpolate.splrep(np.arange(0, len(trajec), 1), trajec)
    trajec = interpolate.splev(ti_new, postck) 

    strajec = scaler.fit_transform(np.reshape(trajec,(-1,1))).T[0]       
    data_set[i] = strajec + namp * convt * np.random.normal(0,1,size=100)

    
tend = time.time()

print('data ready')
print('elapsed time: {0:.0f} min {1:.0f} sec'.format((tend-tstart)//60, (tend-tstart)%60))



data ready
elapsed time: 0 min 8 sec


In [15]:
train_size = 4000
test_size = 1000
f_dim = 40

idxs = np.arange(0, nmax, dtype='int')
np.random.shuffle(idxs)

idx = idxs[:train_size]
idx_test = idxs[train_size:train_size+test_size]

train_data = np.zeros((ncase*train_size,ndiv,1))
train_info = np.zeros((ncase*train_size,f_dim))
train_label = np.zeros((ncase*train_size))

test_data = np.zeros((ncase*test_size,ndiv,1))
test_info = np.zeros((ncase*test_size,f_dim))
test_label = np.zeros((ncase*test_size))



tstart = time.time()

icount = 0
for ic in range(ncase):
    for j, iid in enumerate(idx):
        train_data[icount,:,0] = data_set[ic*nmax+iid]
        train_info[icount] = trajec_feature(data_set[ic*nmax+iid]) 
        train_label[icount] = elabels[ic*nmax + iid]
        icount = icount + 1

icount = 0
for ic in range(ncase):
    for j, iid in enumerate(idx_test):
        test_data[icount,:,0] = data_set[ic*nmax+iid]
        test_info[icount] = trajec_feature(data_set[ic*nmax+iid]) 
        test_label[icount] = elabels[ic*nmax + iid]
        icount = icount + 1

tend = time.time()

print('elapsed time: {0:.0f} min {1:.0f} sec'.format((tend-tstart)//60, (tend-tstart)%60))



elapsed time: 1 min 14 sec


In [16]:
import tensorflow.keras as keras
import tensorflow as tf

keras.backend.clear_session()

machine_id = 0
estop_pat = 50
rlr_pat = 30

epnum = 1000
bsize = 128

l_dim = f_dim

MODEL_SAVE_FOLDER_PATH = './test_build/'
if not os.path.exists(MODEL_SAVE_FOLDER_PATH):
    os.mkdir(MODEL_SAVE_FOLDER_PATH)
    
model_path = MODEL_SAVE_FOLDER_PATH + 'task1_{0:}.hdf5'.format(machine_id)

checkpoint = keras.callbacks.ModelCheckpoint(filepath=model_path, monitor='val_loss', mode = 'min',\
                                verbose=1, save_best_only=True)

stopping = keras.callbacks.EarlyStopping(monitor='val_loss', mode = 'min', patience=estop_pat)
      
rlr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.7, patience=rlr_pat, min_lr=0.000001, \
                        verbose=1, min_delta=1e-5)





In [17]:
# act = 'selu'
# initializer = tf.keras.initializers.LecunNormal()
act = 'relu'
initializer = tf.keras.initializers.HeUniform()

def build_model_resnet(input_shape, l_dim):

    n_feature_maps = 128
#     n_feature_maps = 64
    input_layer = keras.layers.Input(input_shape)

    # BLOCK 1

    conv_x = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=10, strides=1,  \
                                 padding='same', kernel_initializer=initializer)(input_layer)
    conv_x = keras.layers.BatchNormalization()(conv_x)
    conv_x = keras.layers.Activation(act)(conv_x)

    conv_y = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=7, strides=1,  \
                                 padding='same', kernel_initializer=initializer)(conv_x)
    conv_y = keras.layers.BatchNormalization()(conv_y)
    conv_y = keras.layers.Activation(act)(conv_y)

    conv_z = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=5, strides=1,  \
                                 padding='same', kernel_initializer=initializer)(conv_y)
    conv_z = keras.layers.BatchNormalization()(conv_z)

    shortcut_y = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=1, strides=1,  \
                                 padding='same', kernel_initializer=initializer)(input_layer)
    shortcut_y = keras.layers.BatchNormalization()(shortcut_y)

    output_block_1 = keras.layers.add([shortcut_y, conv_z])
    output_block_1 = keras.layers.Activation(act)(output_block_1)

    # BLOCK 2

    conv_x = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=10, strides=1,  \
                                 padding='same', kernel_initializer=initializer)(input_layer)
    conv_x = keras.layers.BatchNormalization()(conv_x)
    conv_x = keras.layers.Activation(act)(conv_x)

    conv_y = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=7, strides=1,  \
                                 padding='same', kernel_initializer=initializer)(conv_x)
    conv_y = keras.layers.BatchNormalization()(conv_y)
    conv_y = keras.layers.Activation(act)(conv_y)

    conv_z = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=5, strides=1,  \
                                 padding='same', kernel_initializer=initializer)(conv_y)
    conv_z = keras.layers.BatchNormalization()(conv_z)

    shortcut_y = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=1, strides=1,  \
                                 padding='same', kernel_initializer=initializer)(output_block_1)
    shortcut_y = keras.layers.BatchNormalization()(shortcut_y)

    output_block_2 = keras.layers.add([shortcut_y, conv_z])
    output_block_2 = keras.layers.Activation(act)(output_block_2)

    # BLOCK 3

    conv_x = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=10, strides=1,  \
                                 padding='same', kernel_initializer=initializer)(output_block_2)
    conv_x = keras.layers.BatchNormalization()(conv_x)
    conv_x = keras.layers.Activation(act)(conv_x)

    conv_y = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=7, strides=1,  \
                                 padding='same', kernel_initializer=initializer)(conv_x)
    conv_y = keras.layers.BatchNormalization()(conv_y)
    conv_y = keras.layers.Activation(act)(conv_y)

    conv_z = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=5, strides=1,  \
                                 padding='same', kernel_initializer=initializer)(conv_y)
    conv_z = keras.layers.BatchNormalization()(conv_z)

    shortcut_y = keras.layers.BatchNormalization()(output_block_2)

    output_block_3 = keras.layers.add([shortcut_y, conv_z])
    output_block_3 = keras.layers.Activation(act)(output_block_3)

    # FINAL

    gap_layer = keras.layers.GlobalAveragePooling1D()(output_block_3)

    output_layer = keras.layers.Dense(l_dim, activation=act)(gap_layer)

    model = keras.models.Model(inputs=input_layer, outputs=output_layer)

    return model


def MLP_read(l_dim, f_dim):

    l_input = keras.layers.Input(shape=(l_dim,))
    f_input = keras.layers.Input(shape=(f_dim,))
    concated = keras.layers.concatenate([l_input, f_input])

    mlp_clf = keras.models.Sequential()
    
    mlp_clf.add(keras.layers.Dense(100, activation='linear', input_dim=l_dim + f_dim, \
                                  kernel_initializer=initializer))
    mlp_clf.add(keras.layers.BatchNormalization())
    mlp_clf.add(keras.layers.Activation(act))
    mlp_clf.add(keras.layers.Dropout(0.2))


    mlp_clf.add(keras.layers.Dense(100, activation='linear', kernel_initializer=initializer))
    mlp_clf.add(keras.layers.BatchNormalization())
    mlp_clf.add(keras.layers.Activation(act))
    mlp_clf.add(keras.layers.Dropout(0.2))


    mlp_clf.add(keras.layers.Dense(100, activation='linear', kernel_initializer=initializer))
    mlp_clf.add(keras.layers.BatchNormalization())
    mlp_clf.add(keras.layers.Activation(act))
    mlp_clf.add(keras.layers.Dropout(0.2))


    mlp_clf.add(keras.layers.Dense(50, activation='linear', kernel_initializer=initializer))
    mlp_clf.add(keras.layers.BatchNormalization())
    mlp_clf.add(keras.layers.Activation(act))
    mlp_clf.add(keras.layers.Dropout(0.2))


    mlp_clf.add(keras.layers.Dense(50, activation='linear', kernel_initializer=initializer))
    mlp_clf.add(keras.layers.BatchNormalization())
    mlp_clf.add(keras.layers.Activation(act))
    mlp_clf.add(keras.layers.Dropout(0.2))

    mlp_clf.add(keras.layers.Dense(50, activation='linear', kernel_initializer=initializer))
    mlp_clf.add(keras.layers.BatchNormalization())
    mlp_clf.add(keras.layers.Activation(act))
    mlp_clf.add(keras.layers.Dropout(0.2))

    mlp_clf.add(keras.layers.Dense(10, activation='linear', kernel_initializer=initializer))
    mlp_clf.add(keras.layers.BatchNormalization())
    mlp_clf.add(keras.layers.Activation(act))
    mlp_clf.add(keras.layers.Dropout(0.2))

    mlp_clf.add(keras.layers.Dense(10, activation='linear', kernel_initializer=initializer))
    mlp_clf.add(keras.layers.BatchNormalization())
    mlp_clf.add(keras.layers.Activation(act))
    mlp_clf.add(keras.layers.Dropout(0.2))

    mlp_clf.add(keras.layers.Dense(10, activation='linear', kernel_initializer=initializer))
    mlp_clf.add(keras.layers.BatchNormalization())
    mlp_clf.add(keras.layers.Activation(act))
    mlp_clf.add(keras.layers.Dropout(0.2))

    mlp_clf.add(keras.layers.Dense(1, activation=act))
    
    out = mlp_clf(concated)

    mlp_model = keras.models.Model([l_input, f_input], out)

    return mlp_model



l_dim = f_dim

input_shape = (ndiv,1,)
res = build_model_resnet(input_shape, l_dim)
res.summary()
keras.utils.plot_model(res, to_file=MODEL_SAVE_FOLDER_PATH +'resnet_t1.png', \
                       show_shapes=True, show_layer_names=True)

mlp_clf = MLP_read(l_dim, f_dim)
mlp_clf.summary()
keras.utils.plot_model(mlp_clf, to_file=MODEL_SAVE_FOLDER_PATH +'mlp_clf_t1.png', \
                       show_shapes=True, show_layer_names=True)

indata = keras.layers.Input(shape=(ndiv,1,))
f_info = keras.layers.Input(shape=(f_dim,))

resnet_out = res(indata)
output = mlp_clf([resnet_out, f_info])


resnet_mlp = keras.models.Model([indata, f_info],output)
resnet_mlp.summary()
keras.utils.plot_model(resnet_mlp, to_file=MODEL_SAVE_FOLDER_PATH +'resnet_mlp_t1.png', \
                       show_shapes=True, show_layer_names=True)


# from keras.utils import plot_model
resnet_mlp.compile(loss='mae', optimizer=keras.optimizers.Adam(0.001))

hist = resnet_mlp.fit([train_data, train_info], train_label, batch_size=bsize, epochs=epnum,
                 verbose=1, validation_data=([test_data, test_info], test_label),\
                 callbacks=[rlr, stopping, checkpoint])     




Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 100, 1)]     0                                            
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 100, 128)     1408        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 100, 128)     512         conv1d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 100, 128)     0           batch_normalization[0][0]        
_______________________________________________________________________________________

KeyboardInterrupt: 

In [ ]:
plt.title('Learning Curves')
plt.xlabel('Epoch')
plt.ylabel('ACC')
plt.plot(hist.history['val_loss'], label='7label')
plt.yscale('log')
plt.savefig( MODEL_SAVE_FOLDER_PATH + 'learning_cur_task1.png')
plt.legend()
plt.show()

In [19]:
loaded = keras.models.load_model(model_path)


###############
file = '../USED_DATA/task_sim_1d_0_nmax5000.pkl'
machine_id = 1
###############


f = open(file, 'rb')
my_list = pickle.load(f)
trajecs = my_list['trajec']
model_id = my_list['label_id']
labels = my_list['label']
elabels = my_list['elabel']
f.flush()
f.close()

ndiv = 100
namp = 0.001

data_set = np.zeros((len(trajecs),ndiv))
scaler = MinMaxScaler(feature_range=(0, 1))

for i in range(len(trajecs)):
    trajec = trajecs[i]
    ret = len(trajecs[i])
    convt = ndiv/ret
    
    ti_new = np.linspace(0, len(trajec), ndiv)
    postck = interpolate.splrep(np.arange(0, len(trajec), 1), trajec)
    trajec = interpolate.splev(ti_new, postck) 

    strajec = scaler.fit_transform(np.reshape(trajec,(-1,1))).T[0]       
    data_set[i] = strajec + namp * convt * np.random.normal(0,1,size=100)



train_size = 3000
test_size = 2000
f_dim = 40

idxs = np.arange(0, nmax, dtype='int')
np.random.shuffle(idxs)
idx = idxs[:train_size]
idx_test = idxs[train_size:train_size+test_size]
train_data = np.zeros((ncase*train_size,ndiv,1))
train_info = np.zeros((ncase*train_size,f_dim))
train_label = np.zeros((ncase*train_size))
test_data = np.zeros((ncase*test_size,ndiv,1))
test_info = np.zeros((ncase*test_size,f_dim))
test_label = np.zeros((ncase*test_size))

tstart = time.time()
icount = 0
for ic in range(ncase):
    for j, iid in enumerate(idx):
        train_data[icount,:,0] = data_set[ic*nmax+iid]
        train_info[icount] = trajec_feature(data_set[ic*nmax+iid]) 
        train_label[icount] = elabels[ic*nmax + iid]
        icount = icount + 1

icount = 0
for ic in range(ncase):
    for j, iid in enumerate(idx_test):
        test_data[icount,:,0] = data_set[ic*nmax+iid]
        test_info[icount] = trajec_feature(data_set[ic*nmax+iid]) 
        test_label[icount] = elabels[ic*nmax + iid]
        icount = icount + 1

tend = time.time()

print('elapsed time: {0:.0f} min {1:.0f} sec'.format((tend-tstart)//60, (tend-tstart)%60))

estop_pat = 100
rlr_pat = 30

epnum = 1000
bsize = 128

l_dim = f_dim

MODEL_SAVE_FOLDER_PATH = './Final_build/'
if not os.path.exists(MODEL_SAVE_FOLDER_PATH):
    os.mkdir(MODEL_SAVE_FOLDER_PATH)
    
model_path = MODEL_SAVE_FOLDER_PATH + 'task1_{0:}.hdf5'.format(machine_id)

checkpoint = keras.callbacks.ModelCheckpoint(filepath=model_path, monitor='val_loss', mode = 'min',\
                                verbose=1, save_best_only=True)

stopping = keras.callbacks.EarlyStopping(monitor='val_loss', mode = 'min', patience=estop_pat)
      
rlr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.7, patience=rlr_pat, min_lr=0.000001, \
                        verbose=1, min_delta=1e-5)


loaded.compile(loss='mae', optimizer=keras.optimizers.Adam())

hist = loaded.fit([train_data, train_info], train_label, batch_size=bsize, epochs=epnum,
                 verbose=1, validation_data=([test_data, test_info], test_label),\
                 callbacks=[rlr, stopping, checkpoint])     



KeyboardInterrupt: 

In [ ]:
plt.title('Learning Curves')
plt.xlabel('Epoch')
plt.ylabel('ACC')
plt.plot(hist.history['val_loss'], label='7label')
plt.yscale('log')
plt.savefig( MODEL_SAVE_FOLDER_PATH + 'learning_cur_task1_1.png')
plt.legend()
plt.show()